# `runSofa` and Scene Configuration
This tutorial explains how to use the `runSofa` executable to launch SOFA simulations and how to write Python scripts that are compatible with both the SOFA graphical user interface and standalone Python execution.

### Learning Objectives
- Understand the role of the `runSofa` executable.
- Learn the `createScene` entry point for SOFA scripts.
- Write hybrid scripts compatible with both `runSofa` and the Python interpreter.
- Configure scenes for different execution environments.

---

## 1. What is `runSofa`?
`runSofa` is the primary executable shipped with SOFA. It is a C++ application that provides a full-featured graphical environment for loading, visualizing, and interacting with SOFA scenes.

While you can run SOFA entirely from a Python script (as seen in previous tutorials), using `runSofa` offers several advantages:
- Built-in scene inspection tools.
- Real-time parameter modification.
- Optimized rendering and recording tools.

### Environment Setup
First, let's import the `Sofa` module.

In [ ]:
import Sofa

## 2. The `createScene` Function
To make a Python script compatible with `runSofa`, you must define a special entry point: the `createScene` function.

When `runSofa` loads a Python file, it:
1. Imports the module.
2. Looks for a function named `createScene`.
3. Calls `createScene(rootNode)`, passing the automatically created **root node** as an argument.

Here is the simplest possible `createScene` function:

In [ ]:
def createScene(rootNode):
    """
    Standard entry point for SOFA scenes.
    :param rootNode: The automatically created root node of the scene graph.
    """
    return rootNode

## 3. Loading Scripts in `runSofa`
To run your script with `runSofa`, use the following command in your terminal:

```bash
runSofa script_name.py
```

### The `SofaPython3` Plugin
For `runSofa` to understand Python files, it must have the `SofaPython3` plugin loaded. In many installations, this is automatic. If not, you can force it using the `-l` (load) flag:

```bash
runSofa -l SofaPython3 script_name.py
```

---

## 4. Writing Hybrid Scripts
A common best practice is to write scripts that can be executed both by `runSofa` (for GUI interaction) and by the Python interpreter (for automated testing or headless processing).

To achieve this, we use the `if __name__ == '__main__':` idiom.

### Example: A Minimal Hybrid Script

In [ ]:
def createScene(rootNode):
    # This part is executed by both runSofa and Python
    rootNode.addObject("DefaultAnimationLoop")
    return rootNode

def main():
    # This part is ONLY executed when running directly via 'python script.py'
    import SofaRuntime
    SofaRuntime.init()

    # Manually create the root node (runSofa does this for us)
    root = Sofa.Core.Node("root")

    # Build the scene using our function
    createScene(root)

    # Initialize and run a few steps
    Sofa.Simulation.initRoot(root)
    for iteration in range(10):
        Sofa.Simulation.animate(root, root.dt.value)
    
    print("Simulation finished successfully in headless mode.")

if __name__ == '__main__':
    main()

## 5. Hybrid Script with GUI Support
You can also include GUI logic in your `main()` function so that running `python script.py` opens a window similar to `runSofa`.

In [ ]:
import Sofa.Gui
import SofaImGui

def createScene(rootNode):
    rootNode.addObject("DefaultAnimationLoop")
    return rootNode

def main():
    import SofaRuntime
    SofaRuntime.init()

    root = Sofa.Core.Node("root")
    createScene(root)
    Sofa.Simulation.initRoot(root)

    # GUI Initialization
    Sofa.Gui.GUIManager.Init("HybridScene", "imgui")
    Sofa.Gui.GUIManager.createGUI(root, "Running via Python CLI")
    Sofa.Gui.GUIManager.SetDimension(1080, 800)

    # Launch the GUI
    Sofa.Gui.GUIManager.MainLoop(root)
    Sofa.Gui.GUIManager.closeGUI()

if __name__ == '__main__':
    main()

### Summary
- **`runSofa`**: The standard C++ executable for SOFA simulations.
- **`createScene(rootNode)`**: The mandatory function for `runSofa` compatibility.
- **Hybrid Approach**: Use `main()` to allow your scripts to run in any environment.

By following this structure, your SOFA components and scenes become highly portable and easy to share across different workflows.